In [ ]:
# pylint: disable=invalid-name,missing-module-docstring

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# absolute path of the input file that contains the msms refs to want to append to
# or None to generate an msms refs file that only contains new entries
input_file_name = "/global/project/projectdirs/metatlas/projects/spectral_libraries/msms_refs_v3.tab"

# Require the input file to pass all validation checks?
# if True, you won't be able to export a file if input_file_name has any rows not passing validation
validate_input_file = False

# number of input rows to initially display
num_rows_to_add = 1

# absolute path of where you want this notebook to store the new MSMS refs
output_file_name = "/global/homes/FIRST-INITIAL-OF-USERNAME/USERNAME/metabolomics_projects/my_msms_refs.tab"

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import logging  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=wjhjgi/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
try:
    with Path("/metatlas_image_version").open(mode="r", encoding="utf-8") as f:
        version = f.readlines()[0].rstrip()
except FileNotFoundError:
    version = "0"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not (version == "1" and (kernel_file_name.is_file() or has_root_kernel)):
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical("CRITICAL: Notebook kernel has been updated. Restart kernel and re-run notebook.")
    raise StopExecution
try:
    from metatlas.tools import notebook  # noqa: E402
    from metatlas.tools import add_msms_ref as amr  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Please check that the kernel is set to "Metatlas Targeted".')
    raise StopExecution from err
notebook.setup(log_level, source_code_version_id)
amr.add_msms_refs(input_file_name, output_file_name, validate_input_file, num_rows_to_add)